In [48]:
import pandas as pd
df=pd.read_csv(r"E:\Stack-Overflow\data\Processed\processed_data.csv")
df.head()

,Title,Body,Tags
0,good branch merge tutorials tortoisesvn,really good tutorials explain branch merge apa...,['svn']
1,asp.net site map,anyone get experience create sqlbased asp.net ...,"['sql', 'asp.net']"
2,function create color wheel,something pseudosolved many time never quite f...,['algorithm']
3,add script functionality .net applications,little game write c use database backend trade...,"['c#', '.net']"
4,use nest class case,work collection class use video playback recor...,"['c++', 'oop', 'class']"


In [49]:
df["Text"]=df["Title"]+" "+df["Body"]


In [50]:
df.head()

,Title,Body,Tags,Text
0,good branch merge tutorials tortoisesvn,really good tutorials explain branch merge apa...,['svn'],good branch merge tutorials tortoisesvn really...
1,asp.net site map,anyone get experience create sqlbased asp.net ...,"['sql', 'asp.net']",asp.net site map anyone get experience create ...
2,function create color wheel,something pseudosolved many time never quite f...,['algorithm'],function create color wheel something pseudoso...
3,add script functionality .net applications,little game write c use database backend trade...,"['c#', '.net']",add script functionality .net applications lit...
4,use nest class case,work collection class use video playback recor...,"['c++', 'oop', 'class']",use nest class case work collection class use ...


In [51]:
df.drop(columns=["Title","Body"],inplace=True)
df.head()



,Tags,Text
0,['svn'],good branch merge tutorials tortoisesvn really...
1,"['sql', 'asp.net']",asp.net site map anyone get experience create ...
2,['algorithm'],function create color wheel something pseudoso...
3,"['c#', '.net']",add script functionality .net applications lit...
4,"['c++', 'oop', 'class']",use nest class case work collection class use ...


In [52]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ytrso\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ytrso\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ytrso\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [53]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    return ' '.join(tokens)


In [54]:
data=df
data.head()

,Tags,Text
0,['svn'],good branch merge tutorials tortoisesvn really...
1,"['sql', 'asp.net']",asp.net site map anyone get experience create ...
2,['algorithm'],function create color wheel something pseudoso...
3,"['c#', '.net']",add script functionality .net applications lit...
4,"['c++', 'oop', 'class']",use nest class case work collection class use ...


In [55]:
print(data['Text'].isnull().sum())  # see if any missing
data['Text'] = data['Text'].fillna("")  # fill missing with empty string


5


In [56]:
import ast

data['Tags'] = data['Tags'].apply(ast.literal_eval)


In [57]:
# Clean text
data['Cleaned_Text'] = data['Text'].apply(preprocess_text)

# Encode tags
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(data['Tags'])

# TF-IDF vectorization
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(data['Cleaned_Text'])

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [58]:
model = OneVsRestClassifier(LogisticRegression(max_iter=1000))
model.fit(X_train, y_train)


OneVsRestClassifier(estimator=LogisticRegression(max_iter=1000))

In [59]:
import joblib
import os

# Create 'models' folder if it doesn't exist
os.makedirs('models', exist_ok=True)

# Assuming these are already trained/fitted:
# model        -> your trained classifier
# vectorizer   -> your fitted TfidfVectorizer
# mlb          -> your fitted MultiLabelBinarizer

# Save each component to 'models' folder
joblib.dump(model, 'models/model.pkl')
joblib.dump(vectorizer, 'models/vectorizer.pkl')
joblib.dump(mlb, 'models/mlb.pkl')

print("✅ Model, vectorizer, and label binarizer saved successfully in 'models' folder.")


✅ Model, vectorizer, and label binarizer saved successfully in 'models' folder.


In [60]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=mlb.classes_))


                    precision    recall  f1-score   support

              .net       0.62      0.14      0.22       527
              ajax       0.81      0.28      0.41       105
         algorithm       0.84      0.30      0.45       161
           android       0.98      0.75      0.85      1082
         angularjs       0.97      0.49      0.65       171
               api       0.67      0.08      0.14        53
            arrays       0.61      0.29      0.39       161
           asp.net       0.77      0.27      0.40       267
       asp.net-mvc       0.78      0.44      0.56       200
     asp.net-mvc-3       0.44      0.06      0.10        71
              bash       0.90      0.38      0.53        98
                 c       0.76      0.29      0.42       346
                c#       0.77      0.39      0.52      1380
               c++       0.86      0.53      0.65       835
             c++11       0.71      0.25      0.37       139
             class       0.33      0.02

e:\Stack-Overflow\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
e:\Stack-Overflow\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [61]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from xgboost import XGBClassifier  # optional
import pandas as pd
import numpy as np

In [ ]:
models = {
    "Logistic Regression": OneVsRestClassifier(LogisticRegression(max_iter=1000)),
    "Random Forest": OneVsRestClassifier(RandomForestClassifier(n_estimators=100)),
    "Linear SVM": OneVsRestClassifier(LinearSVC()),
    "XGBoost": OneVsRestClassifier(XGBClassifier(use_label_encoder=False, eval_metric='logloss'))
}

for name, model in models.items():
    print(f"\n🔍 Training: {name}")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"📊 Classification Report for {name}")
    print(classification_report(y_test, y_pred, target_names=mlb.classes_, zero_division=0))
